In [ ]:
!pip install --upgrade pip
!pip install neptune-client
import neptune

In [ ]:
import torch
import pandas as pd
import matplotlib.image as mpimg
import numpy as np
import cv2
import time
from torch.nn import functional as torch_functional
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import albumentations as A
from albumentations import pytorch as ATorch

#?????????????????????????????????????????
import random
import os


In [ ]:
!pip install --upgrade pip
!pip install -q efficientnet_pytorch
import efficientnet_pytorch

In [ ]:
model_params = {
    "epoch": 15,
    "lr": 0.0001,
    "batch_size": 32,
    "save_path": "./model-best.torch",
    "optimizer": "torch.optim.Adam(model.parameters(), lr=0.0001)",
    "criterion": torch_functional.cross_entropy,
    "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    "patience": 3
}

In [ ]:
model_params["device"]

змінити чітко прописані параметри на параметри зі словника
зберегти код в нептун

In [ ]:
f = open("../input/neptun/Api_Neptun.txt")
my_api = f.read()
project_name = '000flowerprincess000/CassavaLeafDiseaseClassification'
neptune.init(project_qualified_name=project_name, api_token=my_api,)
neptune.create_experiment(name=f"efficientnet-b0_experiment", params = model_params, upload_source_files=os.listdir(os.getcwd()))
f.close()

In [ ]:
efficientnet_pytorch.EfficientNet.from_pretrained("efficientnet-b0")
#This model takes input images of shape (224, 224, 3), and the input data should range [0, 255]. Normalization is included as part of the model.

In [ ]:
# Try to set random seet that our experiment repeated between (We have some problem to set seed with GPU)
def set_seed(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True


set_seed(7)

In [ ]:
#Augumentation

def train_augumentation():
    return A.Compose([
        A.CLAHE(p=0.5, clip_limit=(1, 15), tile_grid_size=(8, 8)),
        A.Cutout(p=0.5, num_holes=50, max_h_size=8, max_w_size=8),
        A.Downscale(p=0.5, scale_min=0.8, scale_max=0.99, interpolation=1),
        A.Flip(p=0.5),
        A.GaussNoise(p=0.5, var_limit=(111.83999633789062, 266.4499816894531)),
        A.HorizontalFlip(p=0.5),
        A.ISONoise(p=0.25, intensity=(0.0, 0.429999977350235), color_shift=(0.10999999940395355, 0.4899999797344208)),
        A.ImageCompression(p=0.25, quality_lower=20, quality_upper=95, compression_type=0),
        A.RandomBrightnessContrast(p=0.25, brightness_limit=(-0.3799999952316284, 0.35999998450279236), contrast_limit=(-0.3400000035762787, 0.38999998569488525), brightness_by_max=True),
        A.RandomGridShuffle(p=0.25, grid=(3, 3)),
        
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], p=1.0),
        ATorch.transforms.ToTensorV2(p=1.0)])

def valid_augumentation():
    return A.Compose([
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], p=1.0),
        ATorch.transforms.ToTensorV2(p=1.0)])

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, index_list, labels_list, augumentation):
        self.labels_list = labels_list[0:100]
        self.index_list = index_list[0:100]
        self.augumentation = augumentation     #cюди передається обєкт класа A.Compose із функції train_augumentation() або valid_augumentation() в залежності від того 
                                               #для якого набору даних цей клас(Dataset) буде використовуватись, тренувального чи валідаційного

    def __len__(self):
        return len(self.labels_list)

    def __getitem__(self, index):
        # Select sample
        image = cv2.imread("../input/cassava-leaf-disease-classification/train_images/" + self.index_list[index])
        label = self.labels_list[index]
        image = cv2.resize(image, (224, 224))
#         image = torch.Tensor(image)           # на даний момент ці рядки замінюють функції A.Normalize і ATorch.transforms.ToTensorV2 у функції агументації
#         image = image.permute(2, 0, 1)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = self.augumentation(image=image)["image"]
        return image, label

In [ ]:
data = pd.read_csv("../input/cassava-leaf-disease-classification/train.csv")
X_train, X_valid, y_train, y_valid = train_test_split(data["image_id"], data["label"], test_size=0.2, random_state=42)

X_train = X_train.reset_index(drop=True)
X_valid = X_valid.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_valid = y_valid.reset_index(drop=True)


train_data = Dataset(X_train, y_train, train_augumentation())
valid_data = Dataset(X_valid, y_valid, valid_augumentation())

In [ ]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.network = efficientnet_pytorch.EfficientNet.from_pretrained("efficientnet-b0")
        self.network._fc = torch.nn.Linear(in_features=1280, out_features=5, bias=True)
    
    def forward(self, data):
        return self.network(data)

In [ ]:
loader_train_data = torch.utils.data.DataLoader(train_data, batch_size=model_params["batch_size"],shuffle=True)
loader_valid_data = torch.utils.data.DataLoader(valid_data, batch_size=model_params["batch_size"],shuffle=False)

In [ ]:
def denormalize_image(image):
    return image * [0.229, 0.224, 0.225] + [0.485, 0.456, 0.406]

# Let's visualize some batches of the train data
fig = plt.figure(figsize=(16, 16))
for i_batch, batch in enumerate(loader_train_data):
    images, labels = batch[0], batch[1]
    count = 1
    for i in range(len(images)):
        plt.subplot(4, 4, count)
        plt.imshow(denormalize_image(images[i].permute(1, 2, 0).numpy()))
        plt.title(labels[i].numpy())
        plt.axis("off")
        count+=1
        if count == 17:
            break
    if i_batch >= 3:
        break
        

neptune.log_image("data-examples", fig)

In [ ]:
model = Model()

In [ ]:
class Trainer:
    def __init__(self, model, device, optimizer, criterion, loss_meter, score_meter):
        self.model = model
        self.device = device
        self.optimizer = optimizer
        self.criterion = criterion
        self.loss_meter = loss_meter
        self.score_meter = score_meter
        
        self.best_valid_score = -np.inf
        self.n_patience = 0
        
        self.messages = {
            "epoch": "[Epoch {}: {}] loss: {:.5f}, score: {:.5f} ☀️",
            "patience": "\nValid score didn't improve last {} epochs.  ❌❌❌"
        }
        
        self.history = {
            "train_loss": [],
            "train_score": [],
            "valid_loss": [],
            "valid_score": [],
        }
    
    def fit(self, epoch, loader_train_data, loader_valid_data, save_path, patience):
        for i in range(epoch):
            self.info_message(f"Epoch {i}")
            
            train_loss, train_score = self.train_epoch(loader_train_data)
            valid_loss, valid_score = self.valid_epoch(loader_valid_data)
            
            self.info_message(self.messages["epoch"], "Train", i, train_loss, train_score)
            self.info_message(self.messages["epoch"], "Valid", i, valid_loss, valid_score)
            
            neptune.log_metric(f'train_loss', train_loss)
            neptune.log_metric(f'train_accuracy', train_score)
            neptune.log_metric(f'valid_loss', valid_loss)
            neptune.log_metric(f'valid_accuracy', valid_score)
                
            if self.best_valid_score < valid_score:
                self.best_valid_score = valid_score
                self.save_model(i, save_path)
                self.n_patience = 0
            else:
                self.n_patience += 1
                
            if self.n_patience >= patience:
                self.info_message(self.messages["patience"], patience)
                break
                
        return self.history
    
    def train_epoch(self, loader_train_data):
        self.model.train()                                                        #переводим модель в режим тренування
        train_loss = self.loss_meter()                                            #створюєм обєкт класа LossMeter 
        train_score = self.score_meter()                                          #створюєм обєкт класа AccMeter
        
        for step, batch in enumerate(loader_train_data, 1):                            #перебираємо батчі
            images = batch[0].to(self.device)                                     #переносимо дані на вказаний девайс
            targets = batch[1].to(self.device)                                    #переносимо дані на вказаний девайс
            self.optimizer.zero_grad()                                            #обнуляє стан оптимайзера
            outputs = self.model(images)                                          #пропускаю дані чеез модель і отримую вихід

            loss = self.criterion(outputs, targets)                               #оцінюєм точність моделі, зрівнюєм передбачення можелі і справжні відповіді
            loss.backward()                                                       #бекпропагейшн

            train_loss.update(loss.detach().item())                               #перераховуєм loss
            train_score.update(targets, outputs.detach())                         #перераховуєм score

            self.optimizer.step()                                                 #обновляєм ваги моделі
            
        self.history["train_loss"].append(train_loss.avg)                              #зберігаєм історію результатів в словник
        self.history["train_score"].append(train_score.avg)                            #зберігаєм історію результатів в словник
        
        return train_loss.avg, train_score.avg                                    #повертаєм значення 
    
    def valid_epoch(self, loader_valid_data):
        self.model.eval()                                                         #переводимо модель в режим оцінювання
    
        valid_loss = self.loss_meter()                                            #створюєм обєкт класа LossMeter
        valid_score = self.score_meter()                                          #створюєм обєкт класа AccMeter

        for step, batch in enumerate(loader_valid_data, 1):                            #перебираємо батчі
            with torch.no_grad():                                                 #вказуємо не зберігати проміжні обчислення так як не будемо робити бекпропагейшн
                images = batch[0].to(self.device)                                 #переносимо дані на вказаний девайс
                targets = batch[1].to(self.device)                                #переносимо дані на вказаний девайс

                outputs = self.model(images)                                      #пропускаю дані чеез модель і отримую вихід
                loss = self.criterion(outputs, targets)                           #оцінюєм точність моделі, зрівнюєм передбачення можелі і справжні відповіді

                valid_loss.update(loss.detach().item())                           #перераховуєм loss
                valid_score.update(targets, outputs)                              #перераховуєм score 
        
        self.history["valid_loss"].append(valid_loss.avg)                              #зберігаєм історію результатів в словник
        self.history["valid_score"].append(valid_score.avg)                            #зберігаєм історію результатів в словник
        
        return valid_loss.avg, valid_score.avg
    
    def save_model(self, n_epoch, save_path):
        torch.save(
            {
                "model_state_dict": self.model.state_dict(),
                "optimizer_state_dict": self.optimizer.state_dict(),
                "best_valid_score": self.best_valid_score,
                "n_epoch": n_epoch,
            },
            save_path,
        )
    
    @staticmethod
    def info_message(message, *args, end="\n"):
        print(message.format(*args), end=end)
        
        
        
        
class LossMeter:
    def __init__(self):
        self.avg = 0
        self.n = 0

    def update(self, val):
        self.n += 1
        # incremental update
        self.avg = val / self.n + (self.n - 1) / self.n * self.avg
        
class AccMeter:
    def __init__(self):
        self.avg = 0
        self.n = 0
        
    def update(self, y_true, y_pred):
        y_true = y_true.cpu().numpy().astype(int)
        y_pred = y_pred.cpu().numpy().argmax(axis=1).astype(int)
        last_n = self.n
        self.n += len(y_true)
        true_count = np.sum(y_true == y_pred)
        # incremental update
        self.avg = true_count / self.n + last_n / self.n * self.avg

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
model = model.to(model_params["device"])
trainer = Trainer(model, model_params["device"], optimizer, model_params["criterion"], LossMeter, AccMeter)

In [ ]:
history = trainer.fit(
    model_params["epoch"], 
    loader_train_data, 
    loader_valid_data, 
    model_params["save_path"], 
    model_params["patience"],
)

In [ ]:
neptune.log_artifact("./model-best.torch")
neptune.stop()

In [ ]:
plt.figure(figsize=(16,6))
plt.subplot(1, 2, 1)
plt.plot(history["train_loss"], label="train_loss")
plt.plot(history["valid_loss"], label="valid_loss")
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(fontsize=15)
plt.xlabel("Epoch number", fontsize=15)
plt.ylabel("Loss value", fontsize=15)
plt.grid()

plt.subplot(1, 2, 2)
plt.plot(history["train_score"], label="train_score")
plt.plot(history["valid_score"], label="valid_score")
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(fontsize=15)
plt.xlabel("Epoch number", fontsize=15)
plt.ylabel("Score value", fontsize=15)
plt.grid()